In [ ]:
from pyModbusTCP.client import ModbusClient

# 建立 Modbus 客戶端
client = ModbusClient(host="127.0.0.1", port=502)


if not client.is_open:
    if not client.open():
        print("無法連接到 Modbus Server")
    else:
        print("已連接到 Modbus Server")


start = 0  # 起始位址
count = 10  # 要讀取的數量

result = client.read_holding_registers(start, count) # 讀取Register
# result = client.read_coils(start, count) # 讀取Coil

if result:
    print("讀取的值：", result)
else:
    print("Modbus 讀取失敗")

client.close()

In [ ]:
from pyModbusTCP.client import ModbusClient

# 建立 Modbus 客戶端
client = ModbusClient(host="127.0.0.1", port=502)

if not client.is_open:
    if not client.open():
        print("無法連接到 Modbus Server")
    else:
        print("已連接到 Modbus Server")

address = 5  # 要寫入的位址
value_to_write = 10  # 要寫入的值


result = client.write_single_register(address, value_to_write) # 寫入Register
# result = client.write_single_coil(address, value_to_write) # 寫入Coil

if result:
    print("寫入成功")
else:
    print("寫入失敗")

client.close()

In [4]:
import scapy.all as scapy
import scapy.contrib.modbus as mb

for pkt in scapy.PcapReader("write.pcapng"):
    if mb.ModbusADUResponse in pkt:
        pkt.show()
        print(pkt)

###[ Loopback ]### 
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 52
     id        = 44293
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0x0
     src       = 192.168.50.197
     dst       = 192.168.50.197
     \options   \
###[ TCP ]### 
        sport     = 502
        dport     = 64011
        seq       = 1236458106
        ack       = 1209433409
        dataofs   = 5
        reserved  = 0
        flags     = PA
        window    = 8442
        chksum    = 0xc156
        urgptr    = 0
        options   = []
###[ ModbusADU ]### 
           transId   = 0x1
           protoId   = 0x0
           len       = 6
           unitId    = 0x1
###[ Write Single Register Response ]### 
              funcCode  = 0x6
              registerAddr= 0x0
              registerValue= 0x1

Loopback / IP / TCP 192.168.50.197:502 > 192.168.50.197:64011 PA / ModbusADUResponse / ModbusPDU06WriteSin